In [19]:
from matplotlib import pyplot as plt

#encoding: utf-8


import lakiaro_to_learning as lk
import random
import os
import numpy as np
from collections      import deque
from keras.models     import Sequential
from keras.layers     import Dense
from keras.optimizers import Adam

class Agent():
    def __init__(self, state_size, action_size):
        self.weight_backup      = "circle.h5"
        self.state_size         = state_size
        self.action_size        = action_size
        self.memory             = deque(maxlen=2000)
        self.learning_rate      = 0.001
        self.gamma              = 0.95
        self.exploration_rate   = 1.0
        self.exploration_min    = 0.01
        self.exploration_decay  = 0.995
        self.brain              = self._build_model()
        

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(32, input_dim=self.state_size, activation='relu'))
        model.add(Dense(32, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer='rmsprop')
        
        #model 매번 새로 만듬
#         if os.path.isfile(self.weight_backup):
#             model.load_weights(self.weight_backup)
#             self.exploration_rate = self.exploration_min
        return model

    def save_model(self):
            self.brain.save(self.weight_backup)

    def act(self, state):
        if np.random.rand() <= self.exploration_rate:
            #print(1)
            rtn = [(np.random.rand(),np.random.rand())]
            return (rtn)
        #print(2)
        act_values = self.brain.predict(state)
        print(act_values)
        #return np.argmax(act_values)
        return act_values


    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def replay(self, sample_batch_size):
        if len(self.memory) < sample_batch_size:
            return
        sample_batch = random.sample(self.memory, sample_batch_size)
        #print(sample_batch)
        for state, action, reward, next_state, done in sample_batch:
            #print(state, action, reward, next_state, done)
            #[[0.41941679]] [(0.08183111271041532, 0.3177256104731947)] 0 [[0.10764589]] False
            target = reward
            if not done:
                #print('predict, nextstate',self.brain.predict(next_state))
                target = reward + self.gamma * np.amax(self.brain.predict(next_state)[0])
            target_f = self.brain.predict(state)
            #print('target_f',target_f)
            #target_f[0]= target
            target_f[0] = target
            self.brain.fit(state, target_f, epochs=1, verbose=0)
            #self.brain.fit(action, next_state, epochs=1, verbose=0)
        if self.exploration_rate > self.exploration_min:
            self.exploration_rate *= self.exploration_decay
class circle():
    
    def __init__(self):
        pass

    
    def reset(self):
        self.score = 0
        self.x=[]
        self.y=[]
        x=0
        y=0
        self.Done = False
        self.index = 0
        return x**2 +y**2
        
    def input_xy(self,xy):
        self.index += 1
        x= xy[0][0]
        y=xy[0][1]
        self.x.append(x)
        self.y.append(y)
        if x**2 +y**2 <= 1 and abs(x)<1 and abs(y)<1 :
            rwd = 1
        else:
            rwd = 0
        if self.index ==100:
            self.Done = True
            
        return x**2 +y**2,rwd,self.Done
    
    def to_graph(self):
        plt.plot(self.x, self.y,'or')
        plt.show()
        print(self.score)
        
class one():
    def __init__(self):
        self.circle = circle()
        self.sample_batch_size = 32
        self.episodes          = 10000
        self.state_size        = 1
        self.action_size       = 2
        self.agent             = Agent(self.state_size, self.action_size)


    def run(self):
        try:
            for index_episode in range(self.episodes):
                state = self.circle.reset()
                state = np.reshape(state, [1, self.state_size])
                #print(state)
                done = False
                index = 0
                reward_a = 0
                while not done:
                    action = self.agent.act(state)
                    #print('action', action)
                    next_state, reward, done = self.circle.input_xy(action)
                    #print(next_state, reward, done)
                    next_state = np.reshape(next_state, [1, self.state_size])
                    self.agent.remember(state, action, reward, next_state, done)
                    state = next_state
                    index += 1
                    reward_a += reward
                print("Episode {}# Score: {}".format(index_episode, reward_a))
                #self.circle.to_graph()
                self.agent.replay(self.sample_batch_size)
        finally:
            self.agent.save_model()

if __name__ == "__main__":
    cartpole = one()
    cartpole.run()



1.3439282838700715 0 False
0.22867976880675675 1 False
0.31779584752850004 1 False
0.25403907378676355 1 False
0.7163170140765018 1 False
1.0453756369839118 0 False
0.531625671243647 1 False
0.3455274507031537 1 False
0.5105792258006958 1 False
1.3950388728716132 0 False
0.9275885202795161 1 False
0.25017744274908876 1 False
0.8441353214258108 1 False
0.5540395971195121 1 False
0.9914053972827668 1 False
1.7786056550210074 0 False
0.4069107940289593 1 False
0.4331556927108691 1 False
0.7735467844923927 1 False
0.27303548268827416 1 False
0.49870779570392454 1 False
1.4993738386137538 0 False
0.4626792940735876 1 False
0.8446071213824382 1 False
1.0340433646607203 0 False
1.470115832385488 0 False
0.25977100018379495 1 False
0.23646015847515828 1 False
0.5101257702149758 1 False
0.0015023753135489208 1 False
0.8772853826827579 1 False
0.699928622679548 1 False
1.023859994608583 0 False
1.6707356743592514 0 False
1.464434515155608 0 False
0.5923785105383403 1 False
1.2690701632134025 0 F

KeyboardInterrupt: 